# BBC Text Representations - Retrieval

**Roll Number:** SE22UARI195

**Tasks:**
1. Generate 20 deterministic queries (15 TF-IDF + 5 negation)
2. Rank TEST documents using cosine similarity
3. Calculate MAP@5, Recall@10, Negation Top-1%
4. Save queries.json and rankings.json
5. Print QUERY_SIGNATURE and RANK_SIGNATURE

---

## 1. Setup & Imports

In [ ]:
# Core libraries
import pandas as pd
import numpy as np
import pickle
import json
import hashlib
from pathlib import Path
from collections import defaultdict

# Sklearn
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# Scipy
from scipy.sparse import load_npz

# NLTK
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Gensim
from gensim.models import Word2Vec

# Progress bar
from tqdm.notebook import tqdm

print("✅ Imports successful!")

In [ ]:
# Configuration
ROLL = "SE22UARI195"
CACHE_DIR = Path("../cache")
MODELS_DIR = Path("../models")
OUTPUTS_DIR = Path("../outputs")
DATA_DIR = Path("../data")

print(f"Roll Number: {ROLL}")
print(f"Cache Directory: {CACHE_DIR}")
print(f"Models Directory: {MODELS_DIR}")
print(f"Outputs Directory: {OUTPUTS_DIR}")

## 2. Load Data & Representations

In [ ]:
print("📂 Loading preprocessed data...\n")

with open(CACHE_DIR / 'train_processed.pkl', 'rb') as f:
    train_df = pickle.load(f)

with open(CACHE_DIR / 'test_processed.pkl', 'rb') as f:
    test_df = pickle.load(f)

print(f"✅ TRAIN: {len(train_df)} documents")
print(f"✅ TEST: {len(test_df)} documents")

# Load master.csv for original text
master_df = pd.read_csv(DATA_DIR / 'master.csv')
print(f"✅ MASTER: {len(master_df)} documents")

In [ ]:
print("\n📊 Loading TF-IDF vectorizer and representations...\n")

# Load TF-IDF vectorizer
with open(MODELS_DIR / 'tfidf_vectorizer.pkl', 'rb') as f:
    tfidf_vectorizer = pickle.load(f)

print(f"✅ TF-IDF vectorizer loaded (vocab size: {len(tfidf_vectorizer.vocabulary_):,})")

# Load representations for retrieval
representations = {}

representations['tfidf'] = {
    'train': load_npz(MODELS_DIR / 'X_train_tfidf.npz'),
    'test': load_npz(MODELS_DIR / 'X_test_tfidf.npz')
}

representations['w2v_ns_tfidf'] = {
    'train': np.load(MODELS_DIR / 'X_train_w2v_sg_ns.npy'),
    'test': np.load(MODELS_DIR / 'X_test_w2v_sg_ns.npy')
}

representations['w2v_hs_tfidf'] = {
    'train': np.load(MODELS_DIR / 'X_train_w2v_sg_hs.npy'),
    'test': np.load(MODELS_DIR / 'X_test_w2v_sg_hs.npy')
}

representations['glove_tfidf'] = {
    'train': np.load(MODELS_DIR / 'X_train_glove.npy'),
    'test': np.load(MODELS_DIR / 'X_test_glove.npy')
}

print(f"✅ All representations loaded")

## 3. Generate Deterministic Queries

In [ ]:
print("\n🔍 Generating 20 deterministic queries...\n")

# Deterministic seed from roll number
SEED = int(hashlib.sha256(ROLL.encode()).hexdigest()[:8], 16)
rng = np.random.default_rng(SEED)

print(f"Deterministic SEED: {SEED}")
print(f"Roll: {ROLL}")

### 3.1 Generate 15 TF-IDF Queries

In [ ]:
print("\n📝 Generating 15 TF-IDF queries...\n")

train_ids = sorted(train_df['id'].tolist())
idx = np.linspace(0, len(train_ids)-1, 15, dtype=int)
picked_ids = [train_ids[(i + (SEED % 7)) % len(train_ids)] for i in idx]

print(f"Selected 15 TRAIN document IDs:")
for i, doc_id in enumerate(picked_ids, 1):
    print(f"  {i:2d}. {doc_id}")

queries_tfidf = []
feature_names = tfidf_vectorizer.get_feature_names_out()

print(f"\nExtracting top-2 TF-IDF terms...\n")

for qid_num, doc_id in enumerate(picked_ids, 1):
    doc_idx = train_df[train_df['id'] == doc_id].index[0]
    train_idx = train_df.index.get_loc(doc_idx)
    
    tfidf_vec = representations['tfidf']['train'][train_idx].toarray().flatten()
    top_indices = np.argsort(tfidf_vec)[-2:][::-1]
    top_terms = [feature_names[i] for i in top_indices if tfidf_vec[i] > 0]
    query_text = ' '.join(top_terms[:2])
    source_label = train_df[train_df['id'] == doc_id]['label'].values[0]
    
    query = {
        'qid': f'Q{qid_num:03d}',
        'source_id': doc_id,
        'query_text': query_text,
        'source_label': source_label,
        'type': 'tfidf'
    }
    
    queries_tfidf.append(query)
    print(f"{query['qid']}: '{query_text}' (label: {source_label})")

print(f"\n✅ Generated {len(queries_tfidf)} TF-IDF queries")

### 3.2 Generate 5 Negation Queries

In [ ]:
print("\n🚫 Generating 5 negation queries...\n")

train_with_toklen = train_df.copy()
train_with_toklen['toklen'] = train_with_toklen['tokens'].apply(len)
short5_df = train_with_toklen.sort_values(['toklen', 'id']).head(5)
short5_ids = short5_df['id'].tolist()

print(f"5 shortest TRAIN documents:")
for i, (idx, row) in enumerate(short5_df.iterrows(), 1):
    print(f"  {i}. {row['id']} (tokens: {row['toklen']}, label: {row['label']})")

negation_rules = {
    'good': 'not good', 'great': 'not great', 'best': 'not best',
    'new': 'not new', 'first': 'not first', 'high': 'not high',
    'will': 'will not', 'can': 'cannot', 'could': 'could not'
}

queries_negation = []

print(f"\nApplying negation transformations...\n")

for qid_num, doc_id in enumerate(short5_ids, 16):
    tokens = train_df[train_df['id'] == doc_id]['tokens'].values[0]
    negated_tokens = tokens.copy()
    negation_applied = False
    
    for i, token in enumerate(tokens):
        if token in negation_rules:
            negated_tokens[i] = negation_rules[token]
            negation_applied = True
            break
    
    if not negation_applied and len(tokens) > 0:
        negated_tokens = ['not'] + tokens
    
    query_text = ' '.join(negated_tokens)
    source_label = train_df[train_df['id'] == doc_id]['label'].values[0]
    
    query = {
        'qid': f'Q{qid_num:03d}',
        'source_id': doc_id,
        'query_text': query_text,
        'source_label': source_label,
        'type': 'neg'
    }
    
    queries_negation.append(query)
    print(f"{query['qid']}: '{query_text[:50]}...'")

print(f"\n✅ Generated {len(queries_negation)} negation queries")

In [ ]:
all_queries = queries_tfidf + queries_negation

print(f"\n📋 Total queries: {len(all_queries)}")
print(f"   TF-IDF: {len(queries_tfidf)}")
print(f"   Negation: {len(queries_negation)}")

with open(OUTPUTS_DIR / 'queries.json', 'w', encoding='utf-8') as f:
    json.dump(all_queries, f, indent=2, ensure_ascii=False)

print(f"\n💾 Saved: {OUTPUTS_DIR / 'queries.json'}")

## 4. Ranking Functions

In [ ]:
def rank_documents_sparse(query_text, test_vectors, tfidf_vec, test_ids, top_k=10):
    """Rank TEST documents (sparse TF-IDF)."""
    query_vec = tfidf_vec.transform([query_text])
    similarities = cosine_similarity(query_vec, test_vectors).flatten()
    top_indices = np.argsort(similarities)[::-1][:top_k]
    return [test_ids[i] for i in top_indices]

def rank_documents_dense(query_text, test_vectors, word_vectors, tfidf_vec, tfidf_vocab, test_ids, top_k=10):
    """Rank TEST documents (dense Word2Vec/GloVe with TF-IDF pooling)."""
    tokens = query_text.lower().split()
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
    
    if hasattr(word_vectors, 'wv'):
        vector_size = word_vectors.wv.vector_size
        vocab = word_vectors.wv
    else:
        vector_size = len(next(iter(word_vectors.values())))
        vocab = word_vectors
    
    if len(tokens) == 0:
        query_vec = np.zeros(vector_size)
    else:
        text = ' '.join(tokens)
        tfidf_vec_query = tfidf_vec.transform([text]).toarray()[0]
        weighted_sum = np.zeros(vector_size)
        total_weight = 0.0
        
        for token in tokens:
            if token in tfidf_vocab:
                tfidf_idx = tfidf_vocab[token]
                tfidf_weight = tfidf_vec_query[tfidf_idx]
                
                if hasattr(vocab, '__contains__'):
                    if token in vocab:
                        word_vec = vocab.get_vector(token) if hasattr(vocab, 'get_vector') else vocab[token]
                        weighted_sum += tfidf_weight * word_vec
                        total_weight += tfidf_weight
                else:
                    if token in vocab:
                        weighted_sum += tfidf_weight * vocab[token]
                        total_weight += tfidf_weight
        
        query_vec = weighted_sum / total_weight if total_weight > 0 else np.zeros(vector_size)
    
    query_vec = query_vec.reshape(1, -1)
    similarities = cosine_similarity(query_vec, test_vectors).flatten()
    top_indices = np.argsort(similarities)[::-1][:top_k]
    return [test_ids[i] for i in top_indices]

print("✅ Ranking functions defined!")

## 5. Rank TEST Documents

In [ ]:
print("\n🔍 Loading models for ranking...\n")

test_ids = test_df['id'].tolist()
rankings = {}

w2v_sg_ns = Word2Vec.load(str(MODELS_DIR / 'w2v_sg_ns.model'))
w2v_sg_hs = Word2Vec.load(str(MODELS_DIR / 'w2v_sg_hs.model'))
print("✅ Word2Vec models loaded")

glove_path = DATA_DIR / 'glove.6B.100d.txt'
glove_embeddings = {}

with open(glove_path, 'r', encoding='utf-8') as f:
    for line in tqdm(f, desc="Loading GloVe", total=400000):
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype='float32')
        glove_embeddings[word] = vector

print(f"✅ GloVe loaded: {len(glove_embeddings):,} vectors")

In [ ]:
print("\nRanking with tfidf...")
rankings['tfidf'] = {}

for query in tqdm(all_queries, desc="tfidf"):
    rankings['tfidf'][query['qid']] = rank_documents_sparse(
        query['query_text'], representations['tfidf']['test'], 
        tfidf_vectorizer, test_ids, top_k=10
    )

print(f"✅ Ranked {len(all_queries)} queries\n")

In [ ]:
print("Ranking with w2v_ns_tfidf...")
rankings['w2v_ns_tfidf'] = {}

for query in tqdm(all_queries, desc="w2v_ns"):
    rankings['w2v_ns_tfidf'][query['qid']] = rank_documents_dense(
        query['query_text'], representations['w2v_ns_tfidf']['test'],
        w2v_sg_ns, tfidf_vectorizer, tfidf_vectorizer.vocabulary_,
        test_ids, top_k=10
    )

print(f"✅ Ranked {len(all_queries)} queries\n")

In [ ]:
print("Ranking with w2v_hs_tfidf...")
rankings['w2v_hs_tfidf'] = {}

for query in tqdm(all_queries, desc="w2v_hs"):
    rankings['w2v_hs_tfidf'][query['qid']] = rank_documents_dense(
        query['query_text'], representations['w2v_hs_tfidf']['test'],
        w2v_sg_hs, tfidf_vectorizer, tfidf_vectorizer.vocabulary_,
        test_ids, top_k=10
    )

print(f"✅ Ranked {len(all_queries)} queries\n")

In [ ]:
print("Ranking with glove_tfidf...")
rankings['glove_tfidf'] = {}

for query in tqdm(all_queries, desc="glove"):
    rankings['glove_tfidf'][query['qid']] = rank_documents_dense(
        query['query_text'], representations['glove_tfidf']['test'],
        glove_embeddings, tfidf_vectorizer, tfidf_vectorizer.vocabulary_,
        test_ids, top_k=10
    )

print(f"✅ Ranked {len(all_queries)} queries\n")
print("\n✅ All rankings complete!")

In [ ]:
with open(OUTPUTS_DIR / 'rankings.json', 'w', encoding='utf-8') as f:
    json.dump(rankings['tfidf'], f, indent=2)

print(f"💾 Saved: {OUTPUTS_DIR / 'rankings.json'}")

## 6. Calculate Retrieval Metrics

In [ ]:
def calculate_map_at_k(queries, rankings, test_df, k=5):
    id2label = dict(zip(test_df['id'], test_df['label']))
    aps = []
    for query in queries:
        qid, source_label = query['qid'], query['source_label']
        ranked_docs = rankings[qid][:k]
        hits, score = 0, 0.0
        for i, doc_id in enumerate(ranked_docs, 1):
            if id2label[doc_id] == source_label:
                hits += 1
                score += hits / i
        total_relevant = sum(1 for label in id2label.values() if label == source_label)
        ap = score / min(k, total_relevant) if total_relevant > 0 else 0.0
        aps.append(ap)
    return np.mean(aps)

def calculate_recall_at_k(queries, rankings, test_df, k=10):
    id2label = dict(zip(test_df['id'], test_df['label']))
    recalls = []
    for query in queries:
        qid, source_label = query['qid'], query['source_label']
        ranked_docs = rankings[qid][:k]
        hits = sum(1 for doc_id in ranked_docs if id2label[doc_id] == source_label)
        total_relevant = sum(1 for label in id2label.values() if label == source_label)
        recall = hits / total_relevant if total_relevant > 0 else 0.0
        recalls.append(recall)
    return np.mean(recalls)

def calculate_negation_top1(queries, rankings, test_df):
    id2label = dict(zip(test_df['id'], test_df['label']))
    neg_queries = [q for q in queries if q['type'] == 'neg']
    if len(neg_queries) == 0:
        return 0.0
    correct = 0
    for query in neg_queries:
        qid, source_label = query['qid'], query['source_label']
        top1_doc = rankings[qid][0]
        if id2label[top1_doc] == source_label:
            correct += 1
    return correct / len(neg_queries)

print("✅ Metric functions defined!")

In [ ]:
print("\n📊 Calculating retrieval metrics...\n")

retrieval_results = {}

for repr_name, repr_rankings in rankings.items():
    print(f"Evaluating {repr_name}...")
    map5 = calculate_map_at_k(all_queries, repr_rankings, test_df, k=5)
    recall10 = calculate_recall_at_k(all_queries, repr_rankings, test_df, k=10)
    neg_top1 = calculate_negation_top1(all_queries, repr_rankings, test_df)
    
    retrieval_results[repr_name] = {
        'map@5': float(map5),
        'recall@10': float(recall10),
        'neg_top1': float(neg_top1)
    }
    
    print(f"  MAP@5: {map5:.4f}, Recall@10: {recall10:.4f}, Neg Top-1%: {neg_top1:.4f}\n")

print("✅ Metrics calculated!")

## 7. Summary & Signatures

In [ ]:
print("\n🎉 Notebook 05: Retrieval - COMPLETE!")
print("\n✅ Generated files:")
print(f"  - {OUTPUTS_DIR / 'queries.json'}")
print(f"  - {OUTPUTS_DIR / 'rankings.json'}")
print("\n✅ Signatures printed above")
print("\nNext steps:")
print("  1. Compile results.json from all notebooks")
print("  2. Run pre-submission validator")
print("  3. Write report.pdf (1-2 pages)")
print("  4. Package submission: SE22UARI195/")